In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## Importing Libraries

In [81]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import os
from sklearn.model_selection import train_test_split 
from skimage import transform

In [82]:
%matplotlib inline

## Dataset path

In [83]:
dataset_path = 'E:\\STUDY\\ACADEMIC\\6th sem\\DL\\Assignment\\Assignment-1_Datasets\\A1-Q2_Dataset\\HumanActionClassification'

classes = glob(dataset_path+'/*')
num_classes = len(classes)
print(num_classes)
labels = np.array([], dtype=np.int)
for each_class in classes:
    each_class = each_class[-1]
    num_files = len(glob((dataset_path+'/%s/*.jp*g') % each_class))
    labels = np.append(labels, np.array([int(each_class)-1]*num_files))
image_file_path = glob(dataset_path+'/*/*.jp*g')
#print(image_file_path)

7


## One-hot encoding labels

In [84]:
one_hot_labels = np.eye(num_classes)[labels]
print(one_hot_labels)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


## Resizing images for memory constrains 

In [85]:
image_transform = (224, 224, 3)

In [86]:
images = np.empty((len(image_file_path), *image_transform))
for i in range(len(image_file_path)):
    img = plt.imread(image_file_path[i])
    transformed_img = transform.resize(img, image_transform[0:2], preserve_range=True)
    images[i] = transformed_img
images /= 255.0

C:\Users\sai19\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


## Input, Target, Weights, Bias initialization

In [87]:
X = tf.placeholder(tf.float32, (None, *image_transform))
y = tf.placeholder(tf.float32, (None, num_classes))

# Kernels
weight1 = tf.Variable(tf.random_normal([3, 3, 3, 64]))
weight2 = tf.Variable(tf.random_normal([3, 3, 64, 128]))
weight3 = tf.Variable(tf.random_normal([3, 3, 128, 256]))

bias1 = tf.Variable(tf.random_normal([64]))
bias2 = tf.Variable(tf.random_normal([128]))
bias3 = tf.Variable(tf.random_normal([256]))

  ##  CNN network  [Convolution, Relu and Pooling block and 1 Fully connected block]

In [88]:
#layer 1
conv2d_1 = tf.nn.conv2d(X, weight1, [1, 2, 2, 1], 'VALID') + bias1
relu_1 = tf.nn.relu(conv2d_1)
maxpool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
#layer 2
conv2d_2 = tf.nn.conv2d(maxpool_1, weight2, [1, 2, 2, 1], 'VALID') + bias2
relu_2 = tf.nn.relu(conv2d_2)
maxpool_2 = tf.nn.max_pool(relu_2, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
#layer 3
conv2d_3 = tf.nn.conv2d(maxpool_2, weight3, [1, 2, 2, 1], 'VALID') + bias3
relu_3 = tf.nn.relu(conv2d_3)
maxpool_3 = tf.nn.max_pool(relu_3, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')

#fully connected layer
flatten_layer = tf.layers.Flatten()(maxpool_3)

output = tf.layers.Dense(num_classes)(flatten_layer)


  ##  Loss function and Optimizer

In [89]:
#loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=output))
#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.003)
train = optimizer.minimize(loss)

predicted_ouptut = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
acc = tf.reduce_mean(tf.cast(predicted_ouptut, tf.float32))

In [90]:
#initializing variables
init = tf.global_variables_initializer()

## Number of Epochs and Batch Size

In [91]:
epochs = 200
batch_size = 64

## Shuffling dataset

In [92]:
random_indexes = np.random.randint(0, len(image_file_path), (len(image_file_path)))

In [93]:
images = images[random_indexes]

## Split Train and test dataset

In [94]:
x_train, x_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.25)

In [95]:
del images
del labels
del image_file_path

In [96]:
    sess = tf.InteractiveSession()

C:\Users\sai19\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## Training

In [ ]:
train_loss = []
train_accuracy = []
sess.run(init)
for i in range(epochs):
    for batch in range(len(x_train)//batch_size):
        batch_x = x_train[batch*batch_size:min((batch+1)*batch_size,len(x_train))]
        batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]
        _ = sess.run(train, feed_dict={X:batch_x,y:batch_y})
        loss_, accuracy = sess.run([loss, acc], feed_dict={X: batch_x,y: batch_y})
    print('epoch '+str(i+1)+', Loss '+'{:.6f}'.format(loss_)+', Training Accuracy '+'{:.5f}'.format(accuracy))
    train_loss.append(loss_)
    train_accuracy.append(accuracy)

epoch 1, Loss -1199386.875000, Training Accuracy 0.14062
epoch 2, Loss -6742236.000000, Training Accuracy 0.14062
epoch 3, Loss -27947736.000000, Training Accuracy 0.14062
epoch 4, Loss -96593536.000000, Training Accuracy 0.14062
epoch 5, Loss -285893984.000000, Training Accuracy 0.14062
epoch 6, Loss -739860608.000000, Training Accuracy 0.14062
epoch 7, Loss -1712280576.000000, Training Accuracy 0.15625
epoch 8, Loss -3608528384.000000, Training Accuracy 0.14062
epoch 9, Loss -7030492672.000000, Training Accuracy 0.14062
epoch 10, Loss -12777943040.000000, Training Accuracy 0.14062
epoch 11, Loss -21901180928.000000, Training Accuracy 0.12500
epoch 12, Loss -35691388928.000000, Training Accuracy 0.12500
epoch 13, Loss -55745785856.000000, Training Accuracy 0.17188
epoch 14, Loss -83911262208.000000, Training Accuracy 0.14062
epoch 15, Loss -122292027392.000000, Training Accuracy 0.17188
epoch 16, Loss -173274628096.000000, Training Accuracy 0.14062
epoch 17, Loss -239510945792.000000,

## Plotting the loss and accuracy for training data

In [ ]:
X = np.arange(1, epochs+1)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim((0, 200.0))
plt.plot(X, train_loss)
plt.title('Training Data')

In [ ]:
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(X, train_accuracy)
plt.title('Training Data')

## Test data loss and accuracy

In [ ]:
test_loss, test_accuracy = sess.run([loss, acc], feed_dict={X:x_test[:], y:y_test[:]})

In [ ]:
print('Test loss: %f accuracy: %f' % (test_loss, test_accuracy))

In [ ]:
sess.close()